In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_info = "Resources/schools_complete.csv"
student_info = "Resources/students_complete.csv"
school_df = pd.read_csv(school_info)
student_df = pd.read_csv(student_info)

In [3]:
#merged data for the two data sets as required for the continuation of the assignment
school_data = pd.merge(student_df, school_df, how="left", on=["school_name"])

In [4]:
#found total number of students in district
count_students = len(school_data['student_name'])

#found total number of students passing math
math_grades = school_data['math_score']
passing_math_count = math_grades[math_grades >= 70].count()

#found total number of students passing reading
reading_grades = school_data['reading_score']
passing_reading_count = reading_grades[reading_grades >= 70].count()

#found total number of students passing both math and reading
passedboth_df = school_data.loc[(school_data['reading_score']>=70) & (school_data['math_score']>= 70)]

#found the total budget of the school district
totalbudget = sum(school_df['budget'])
formatedbudget = f'${totalbudget:,}'

#cleaned data to match assignment requirments
average_reading = round(school_data['reading_score'].mean(), 6)
average_math = round(school_data['math_score'].mean(), 6)
passedboth_count = len(passedboth_df)
number_of_schools = len(school_df)


#print(count_students)
#print(average_math)
#print(average_reading)
#print(number_of_schools)
#print(formatedbudget)

In [5]:
#found the district percent passing information

percent_passing_math = round((passing_math_count/count_students)*100, 6)
percent_passing_reading = round((passing_reading_count/count_students)*100, 6)
percent_passing_both = round((passedboth_count/count_students)*100, 6)

#print(percent_passing_math)
#print(percent_passing_reading)
#print(percent_passing_both)

In [6]:
#% assemble the gathered data into a single dataframe

columns = {'Number of Schools': [number_of_schools], 'Total Students': [count_students],'Total Budget': [formatedbudget],
'Average Math Score': [average_math], 'Average Reading Score': [average_reading], '% Passing Math': [percent_passing_math],
'% Passing Reading': [percent_passing_reading],'% Overall Passing': [percent_passing_both]}
District_Summary = pd.DataFrame(data=columns, index = [''])

District_Summary

,Number of Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,15,39170,"$24,649,428",78.985371,81.87784,74.980853,85.805463,65.172326


In [7]:
#created a table incoluding avarage math and reading schools and othe school info
school_average_info= school_data.groupby(["school_name"]).mean()
#school_average_info

In [8]:
#found type for each school
school_types = school_df.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [8]:
# merging type and general data into one data frame to prepare the creation of the larger "Summary table" 
# at the end of this section
school_summary1 = pd.merge(school_average_info,school_types, how="outer", on=["school_name"])

In [9]:
#found the budget of each school
school_budget=school_df.set_index(["school_name"])["budget"]
#school_budget

In [10]:
#found total number of students per school
total_students = school_data["school_name"].value_counts()
#total_students

In [11]:
#found the budget of each shool for each student
budget_per_student = school_budget / total_students
Budget_per_student_df= pd.DataFrame({'school_name':budget_per_student.index,'Budget/Student':budget_per_student.values})
budget_per_student_df = Budget_per_student_df.set_index(["school_name"])

#budget_per_student_df

In [12]:
#merging all data gathered so far to prepare the creation of the larger "Summary table"

school_summary2 = pd.merge(school_summary1,budget_per_student_df, how="outer", on=["school_name"])

#organized the table based on assignment requirments and renamed it to match sample table provided in instructions

organized_school_summary2 = school_summary2[["type","size", "budget","Budget/Student","math_score","reading_score"]]

renamed_school_summary2 = organized_school_summary2.rename(columns={"type":"School Type","size":"Total Students", "budget":"Total School Budget","Budget/Student":"Budget Per Student","math_score":"Average Math Score","reading_score":"Average Reading Score"})

#  Made a concious choice to use Budget per Student instead of "Per Student Budget" as the wording 
#  is more descriptive and conveys the meaning of the data better

In [13]:
#find all students based on passing grade for math and reading
passmath_df = school_data[(school_data["math_score"]>=70)] 
passreading_df = school_data[(school_data["reading_score"]>= 70)]

#set up a data fram with percent of students passing math
school_passingmath = passmath_df.groupby(["school_name"]).count()["student_name"]
percentpassingmath = (school_passingmath / total_students)*100

Percentpassingmath_df = pd.DataFrame({'school_name':percentpassingmath.index,'% Passing Math':percentpassingmath.values})
percentpassingmath_df = Percentpassingmath_df.set_index(["school_name"])

In [14]:
#set up a data fram with percent of students passing reading
school_passingreading = passreading_df.groupby(["school_name"]).count()["reading_score"]
percentpassingreading = (school_passingreading / total_students)*100

Percentpassingreading_df = pd.DataFrame({'school_name':percentpassingreading.index,'% Passing Reading':percentpassingreading.values})
percentpassingreading_df = Percentpassingreading_df.set_index(["school_name"])

In [15]:
#merging the data frames related to passing math an reading to prepare to merge to larger Summary table
passing_students = pd.merge(percentpassingmath_df,percentpassingreading_df, how="outer",on=["school_name"])

#passing_students

In [16]:
# merging passing graded with the other data to prepare for the "Summary table"

summary3 = pd.merge(renamed_school_summary2,passing_students, how="outer",on=["school_name"])

In [17]:
#set up a data fram with percent of students passing both math and reading

passingboth_df = school_data[(school_data["reading_score"]>= 70)&(school_data["math_score"]>=70)]
school_passingboth = passingboth_df.groupby(["school_name"]).count()["student_name"]
percentpassingboth = (school_passingboth / total_students)*100

Percentpassingboth_df = pd.DataFrame({'school_name':percentpassingboth.index,'% Overall Passing':percentpassingboth.values})
percentpassingboth_df = Percentpassingboth_df.set_index(["school_name"])

In [18]:
# Merged partial summary3 dataframe with the overall passing data frame to create final
# School Summary table, and formatted the budget to have dollar sign

School_Summary_Table= pd.merge(summary3,percentpassingboth_df, how="outer",on=["school_name"])
School_Summary_Table_Noformat = pd.merge(summary3,percentpassingboth_df, how="outer",on=["school_name"])

School_Summary_Table["Total School Budget"] = School_Summary_Table["Total School Budget"].map("${:,.2f}".format)
School_Summary_Table["Budget Per Student"] = School_Summary_Table["Budget Per Student"].map("${:,.2f}".format)
School_Summary_Table['Total Students'] = School_Summary_Table['Total Students'].map("{:.0f}".format)
School_Summary_Table

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [22]:
# Top Performing Schools (By % Overall Passing)

Top_Performing_Schools = School_Summary_Table.sort_values(["% Overall Passing"], ascending=False)
Top_Performing_Schools.head(5)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [23]:
#  Buttom Performing Schools (By % Overall Passing)


Buttom_Performing_Schools = School_Summary_Table.sort_values(["% Overall Passing"])
Buttom_Performing_Schools.head(5)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [24]:
#create a data frame for each grade

ninth_df = school_data[(school_data['grade'] == "9th")]
tenth_df = school_data[(school_data["grade"] == "10th")]
eleventh_df = school_data[(school_data['grade'] == "11th")]
twelfth_df = school_data[(school_data['grade'] == "12th")]
#tenth_df.head()

In [25]:
#use the grade dataframes to group students by school and average of math scores
grouped_ninth = ninth_df.groupby(["school_name"]).mean()["math_score"]
grouped_tenth = tenth_df.groupby(["school_name"]).mean()["math_score"]
grouped_eleventh = eleventh_df.groupby(["school_name"]).mean()["math_score"]
grouped_twelfth = twelfth_df.groupby(["school_name"]).mean()["math_score"]

In [26]:
#created a dataframe for average math scores per grade

Math_scores_grade = pd.DataFrame({'9th': grouped_ninth, '10th': grouped_tenth,'11th': grouped_eleventh,
'12th': grouped_twelfth})
Math_scores_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [27]:
#use the grade dataframes to group students by school and average of reading scores
grouped_ninth_r = ninth_df.groupby(["school_name"]).mean()["reading_score"]
grouped_tenth_r = tenth_df.groupby(["school_name"]).mean()["reading_score"]
grouped_eleventh_r = eleventh_df.groupby(["school_name"]).mean()["reading_score"]
grouped_twelfth_r = twelfth_df.groupby(["school_name"]).mean()["reading_score"]

In [28]:
#created a dataframe for average reading scores per grade

Reading_scores_grade = pd.DataFrame({'9th': grouped_ninth_r, '10th': grouped_tenth_r,'11th': grouped_eleventh_r,
'12th': grouped_twelfth_r})
Reading_scores_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [29]:
#created bins and ranges for Score by School Spending

bins = [578.0, 585, 630, 645, 680]
spending_range = ["<$585", "$585-630", "$630-645", "$645-680"]
School_Summary_Table_Noformat["Spending Ranges (Per Student)"] = pd.cut(School_Summary_Table_Noformat["Budget Per Student"], bins, labels=spending_range, include_lowest=True)

#created table for schools spending vs average score and passing scores

Score_by_school_spending = School_Summary_Table_Noformat.groupby(["Spending Ranges (Per Student)"]).mean()
Score_by_school_spending = Score_by_school_spending[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
Score_by_school_spending = Score_by_school_spending.round(decimals = 2)
Score_by_school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


In [30]:
#created bins and ranges for Score by School Size

schoolsize_bins = [0,1000,2000,5000]
schoolsize_range = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
School_Summary_Table_Noformat["School Size"] = pd.cut(School_Summary_Table_Noformat["Total Students"], schoolsize_bins, labels=schoolsize_range, include_lowest=True)

#created table for schools szie vs average score and passing scores

Score_by_school_size = School_Summary_Table_Noformat.groupby(["School Size"]).mean()
Score_by_school_size = Score_by_school_size[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
Score_by_school_size = Score_by_school_size.round(decimals = 2)
Score_by_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


In [31]:
#created table for schools type vs average score and passing scores

Score_by_school_Type = School_Summary_Table_Noformat.groupby(["School Type"]).mean()
Score_by_school_Type = Score_by_school_Type[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
Score_by_school_Type = Score_by_school_Type.round(decimals = 2)
Score_by_school_Type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


# In this analysis we look at the state wide scores for students in math, reading an the overall total
# The analysis looks at: 
- Highest-Performing Schools (by % Overall Passing)
- Lowest-Performing Schools (by % Overall Passing)
- Scores by School Spending
- Scores by School Size
- Scores by School Type

# Summary of findings:
- larger spending doeas not necessarily mean better grades as shown by spending amounts per students table
- the top 5 scoring schools are charter schools where as the buttom 5 are district/public schools 
- the 1000-2000 medium range schools and small schools seem to perfom much better than the 2000+ student schools, most likely due to better attention given to each individual student when the school is not over populated 
- the table for school type vs score reaffirmes that the highest scoring schools are charter schools by 36.8% which is a very large gap 